<a href="https://colab.research.google.com/github/tluxxx/weekly-pattern-in-stock-markets/blob/main/weekly_patterns_(part_1_16_week_cycle_with_pattern_according_to_Gebert).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploiting weekly patterns in stock market for trading
#### part 1: testing a trading pattern in a 16 weeks cycle

This series is based on a strategy-idea originally generated by Thomas Gebert (Kurzfrist-Strategien fuer Anleger, Boersenbuchverlag, 2020 (in German)). The original strategy was tested and expanded. Further modifications are investigated. The this part contains:

*   implementing and testing of the original strategy during the years [2000-2018]
*   expanding of the original strategy to the years after publication of the original study [2019-2024]
*   modification of the original strategy by shifting the trading time
*   including realistic fees into the analyses
*   analyzing key statistical parameters
*   visualisation of the results
*   sensitivity analyses





#1. Preparation & Data Upload

In [1]:
# preparation
from google.colab import drive
drive.mount("/content/gdrive")
# adding the current colab-directory
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/weekly_pattern')


Mounted at /content/gdrive


In [2]:
# importing all standard modules
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
from tabulate import tabulate

In [3]:
# importing customized modules
from helpers_pattern_01 import *

In [ ]:
# re-importing customized modules (if required)
import importlib
import helpers_pattern_01
importlib.reload(helpers_pattern_01)
from helpers_pattern_01 import *

In [4]:
# direct download price dataframe from yfinance
end_date = '2024-03-30'
start_date = '2000-01-01'
price = yf.download('^GDAXI', start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


# 2. Original Strategy by Gebert (slightly simplified)
### Features:  
*   16-weeks cycle
*   tradetime for opening positions (buy/short) & closing positions (sell/cover): each monday 17:30 (at close)

### Modifications:
*   Original Strategy: If first trading day of a week is not Monday, trading on the last trading day of the previous week
*   Proposed Simplification: if first trading day of a week is not Monday, trading on the first trading day of that particular week


In [5]:
# original gebert pattern and setting other parameters
pos_orig = [0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 0, 1, 1, 1, -1]
shift_weeks = 12
# transforming daily into weekly data and calculting weekly trading positions
price_w = transforming_daily_weekly(price, mode='close')
price_w_orig = price_w[price_w.index < '2018-12-31']
pos = pos_weekly(price_w_orig, pos_orig, shift_weeks)

In [6]:
# check if 2018-11-12 is in week_type 4
week_type = (price_w_orig['week_nb'] + shift_weeks) % len(pos_orig)
week_type[week_type.index=='2018-11-12']

start_w
2018-11-12    4
Name: week_nb, dtype: int64

In [8]:
# calculation and plottint/printing of pnl time-series and final results for the [2000-2018] period
main_title = 'PnL for 16-weeks-cycle, original pattern (w/o fees) vs. Buy&Hold'
sub_title = 'instrument: DAX [2000-2018]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

# calculation
pnl_orig = pnl_accumulation(price_w_orig,
                            pos_weekly(price_w_orig, pos_orig, shift_weeks))
pnl_buh_orig = price_w_orig['price'] / price_w_orig['price'][0]

# plotting
fig = go.Figure()
fig.add_trace(go.Scatter(x=pnl_orig.index, y=pnl_orig, name='original pattern'))
fig.add_trace(go.Scatter(x=pnl_buh_orig.index, y=pnl_buh_orig, name='Buy & Hold'))
fig.update_layout(title=title)
fig.update_layout(autosize=False,width=1000,height=600)
fig.update_layout(template = 'plotly_dark', xaxis_title="Date", yaxis_title='PnL')
fig.show()

print(f' *****************************************************************************')
print(f'Summary of results:  Buy & Hold : {pnl_buh_orig[-1]:.2f},   Strategy: {pnl_orig[-1]:.2f}')

 *****************************************************************************
Summary of results:  Buy & Hold : 1.54,   Strategy: 23.56


In [10]:
# calculation and plottint/printing of pnl time-series and final results for the [2000-2024] period
main_title = 'PnL for 16-weeks-cycle, original pattern (w/o fees) vs. Buy&Hold'
sub_title = 'instrument: DAX [2000-2024]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

# calculation
pnl_full = pnl_accumulation(price_w,
                            pos_weekly(price_w, pos_orig, shift_weeks))
pnl_buh_full = price_w['price'] / price_w['price'][0]

# plotting
fig = go.Figure()
fig.add_trace(go.Scatter(x=pnl_full.index, y=pnl_full, name='original pattern'))
fig.add_trace(go.Scatter(x=pnl_buh_full.index, y=pnl_buh_full, name='Buy & Hold'))
fig.update_layout(title=title)
fig.update_layout(autosize=False,width=1000,height=600)
fig.update_layout(template = 'plotly_dark', xaxis_title="Date", yaxis_title='PnL')
fig.show()

print(f' *****************************************************************************')
print(f'Summary of results:  Buy & Hold : {pnl_buh_full[-1]:.2f},   Strategy: {pnl_full[-1]:.2f}')

 *****************************************************************************
Summary of results:  Buy & Hold : 2.71,   Strategy: 26.83


In [9]:
# calculation and plotting/printing of pnl time-series and final results for the [2019-2024] period
# (i.e. after the publication in 2019)
main_title = 'PnL for 16-weeks-cycle, original pattern (w/o fees) vs. Buy&Hold'
sub_title = 'instrument: DAX [2019-2024]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

# calculation
price_w_rec = price_w[price_w.index>'2019-01-01']
pnl_rec = pnl_accumulation(price_w_rec,
                           pos_weekly(price_w_rec, pos_orig, shift_weeks))
pnl_buh_rec = price_w_rec['price'] / price_w_rec['price'][0]

# plotting
fig = go.Figure()
fig.add_trace(go.Scatter(x=pnl_rec.index, y=pnl_rec, name='original pattern'))
fig.add_trace(go.Scatter(x=pnl_buh_rec.index, y=pnl_buh_rec, name='Buy & Hold'))
fig.update_layout(title=title)
fig.update_layout(autosize=False, width=1000, height=600)
fig.update_layout(template = 'plotly_dark', xaxis_title="Date", yaxis_title='PnL')
fig.show()

print(f' *****************************************************************************')
print(f'Summary of results:  Buy & Hold : {pnl_buh_rec[-1]:.2f},   Strategy: {pnl_rec[-1]:.2f}')

 *****************************************************************************
Summary of results:  Buy & Hold : 1.73,   Strategy: 1.16


# 3. Modification of simplified Original Strategy

*   identical to the slightly modified Original Strategy of Gebert (except tradetime)
*   tradetime for opening positions (buy/short) & closing positions (sell/cover): each monday 9:00 (at open)



In [11]:
# transforming daily into weekly data and calculting weekly trading positions
price_w = transforming_daily_weekly(price, mode='open')
pos = pos_weekly(price_w, pos_orig, shift_weeks)

In [13]:
# check if 2018-11-12 is in week_type 4
week_type = (price_w['week_nb'] + shift_weeks) % len(pos_orig)
week_type[week_type.index=='2018-11-12']

start_w
2018-11-12    4
Name: week_nb, dtype: int64

In [16]:
# calculation and plottint/printing of pnl time-series and final results for the [2000-2024] period
main_title = 'PnL for 16-weeks-cycle, original pattern (w/o fees) vs. Buy&Hold'
sub_title = 'instrument: DAX [2000-2024], modified system (trading at open)'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

# calculation
pnl_full = pnl_accumulation(price_w,
                            pos_weekly(price_w, pos_orig, shift_weeks))
pnl_buh_full = price_w['price'] / price_w['price'][0]

# plotting
fig = go.Figure()
fig.add_trace(go.Scatter(x=pnl_full.index, y=pnl_full, name='original pattern'))
fig.add_trace(go.Scatter(x=pnl_buh_full.index, y=pnl_buh_full, name='Buy & Hold'))
fig.update_layout(template = 'plotly_dark', autosize=False,width=1000,height=600)
fig.update_layout(title=title, xaxis_title="Date", yaxis_title='PnL')
fig.show()

print(f' *****************************************************************************')
print(f'Summary of results:  Buy & Hold : {pnl_buh_full[-1]:.2f},   Strategy: {pnl_full[-1]:.2f}')

 *****************************************************************************
Summary of results:  Buy & Hold : 2.62,   Strategy: 31.26


In [17]:
# calculation and plotting/printing of pnl time-series and final results for the [2019-2024] period
# (i.e. after the publication in 2019)
main_title = 'Equity for 16-weeks-cycle, original pattern (w/o fees) vs. Buy&Hold'
sub_title = 'instrument: DAX [2019-2024], modified system (trade at open)'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

# calculation
price_w_rec = price_w[price_w.index>'2019-01-01']
pnl_rec = pnl_accumulation(price_w_rec,
                           pos_weekly(price_w_rec, pos_orig, shift_weeks))
pnl_buh_rec = price_w_rec['price'] / price_w_rec['price'][0]

# plotting
fig = go.Figure()
fig.add_trace(go.Scatter(x=pnl_rec.index, y=pnl_rec, name='original pattern'))
fig.add_trace(go.Scatter(x=pnl_buh_rec.index, y=pnl_buh_rec, name='Buy & Hold'))
fig.update_layout(title=title)
fig.update_layout(autosize=False, width=1000, height=600)
fig.update_layout(template = 'plotly_dark', xaxis_title="Date", yaxis_title='PnL')
fig.show()

print(f' *****************************************************************************')
print(f'Summary of results:  Buy & Hold : {pnl_buh_rec[-1]:.2f},   Strategy: {pnl_rec[-1]:.2f}')

 *****************************************************************************
Summary of results:  Buy & Hold : 1.74,   Strategy: 1.36


In [20]:
# export of results to excel file
with pd.ExcelWriter(r'/content/gdrive/MyDrive/Colab Notebooks/weekly_pattern/16_weeks_cycle (data).xlsx') as writer:
     price.to_excel(writer, sheet_name='daily data')
     price_w.to_excel(writer, sheet_name='weekly data')
     pos.to_excel(writer, sheet_name='Positions')
     pnl_full.to_excel(writer, sheet_name='PnL')

# 4. Trade Statistics

In [22]:
# simulation of real conditions (incl. fee) and calculation of statistics
start_equity = 10000
fee = 0.0025                                                  # 0,25% of the traded volume
fixed_fee = 4.9                                               # 4,90 EUR pro trade
trades = trading_journal(price_w, pos)                        # generation list of trades
trade_key_parameters(trades, price_w)                         # key parameter independent from start_equity
trade_key_parameters2(trades, start_equity, fee, fixed_fee)   # key parameter depending on start_equity

********************************************************
start date:  2000-01-03
end date:    2024-03-25
┌───────────────────────────┬──────────────┬────────┬─────────┬────────┐
│  Parameter                │   All Trades │   LONG │   SHORT │   FLAT │
├───────────────────────────┼──────────────┼────────┼─────────┼────────┤
│ duration (days):          │     8,848.00 │        │         │        │
├───────────────────────────┼──────────────┼────────┼─────────┼────────┤
│ nb of trades:             │       317.00 │  80.00 │  237.00 │        │
├───────────────────────────┼──────────────┼────────┼─────────┼────────┤
│ duration of trades (%):   │       100.00 │  18.77 │   18.75 │  62.48 │
├───────────────────────────┼──────────────┼────────┼─────────┼────────┤
│ percent profitable (%):   │        58.36 │  71.25 │   54.01 │        │
├───────────────────────────┼──────────────┼────────┼─────────┼────────┤
│ profit factor:            │         2.42 │        │         │        │
├──────────────────

In [50]:
# real equity curves (w/o fee and incl fee)
main_title = 'Equity for 16-weeks-cycle, original pattern,  "w/o fees" vs. "incl. fee"'
sub_title = 'instrument: DAX [2000-2024], modified system (trade at open)'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

# caclulcations
eqs = pnl_acc_real_equity(price_w,
                          pos_weekly(price_w, pos_orig, shift=shift_weeks),
                          start_equity, fee, fixed_fee)

# plotting/printing
fig = go.Figure()
fig.add_trace(go.Scatter(x=price_w.index, y=eqs, name='original pattern with fee'))
fig.add_trace(go.Scatter(x=pnl_full.index, y=pnl_full * start_equity, name='original pattern w/o fee'))
fig.add_trace(go.Scatter(x=pnl_buh_full.index, y=pnl_buh_full * start_equity, name='Buy & Hold'))
fig.update_layout(title=title)
fig.update_layout(autosize=False, width=1000, height=600)
fig.update_layout(template = 'plotly_dark', xaxis_title="Date", yaxis_title='Equity')
fig.show()

print(f' *****************************************************************************')
print(f' Equity development (in EUR):  Start Value {start_equity:,.1f}    End Value (w/o fees): {pnl_full[-1]*start_equity:,.1f}   End Value incl. fees: {eqs[-1]:,.1f}')

 *****************************************************************************
 Equity development (in EUR):  Start Value 10,000.0    End Value (w/o fees): 312,593.7   End Value incl. fees: 56,846.3


In [49]:
# distribution of trade-returns
main_title = 'Distribution of returns 16-weeks-cycle, original pattern (w/o fees)'
sub_title = 'instrument: DAX [2000-2024]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = px.histogram(trades, x=['returns'], color='type', nbins= 40)
fig.update_layout(title=title, xaxis_title='Date', yaxis_title='# occurence')
fig.update_layout(template = 'plotly_dark', autosize=False,width=600,height=300)
fig.show()

x1, x2, x3 = trades['returns'].mean()*100, trades[trades['type']=='LONG']['returns'].mean()*100, trades[trades['type']=='SHORT']['returns'].mean()*100
print(f' **************************************************************')
print(f'average returns: {x1:.1f}%  average returns (LONG): {x2:.1f} %, average returns (SHORT) {x3:.1f}% ')


 **************************************************************
average returns: 1.2%  average returns (LONG): 2.5 %, average returns (SHORT) 0.7% 


In [48]:
# distribution of trade-returns
main_title = 'Distribution of returns 16-weeks-cycle, original pattern (w fees)'
sub_title = 'instrument: DAX [2000-2024]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

trades['eqs_chg'] = trades['equity'].pct_change()
fig = px.histogram(trades, x=['eqs_chg'], color='type', nbins= 40)
fig.update_layout(title=title, xaxis_title='Date', yaxis_title='# occurence')
fig.update_layout(template = 'plotly_dark', autosize=False,width=600,height=300)
fig.show()

x1, x2, x3 = trades['eqs_chg'].mean()*100, trades[trades['type']=='LONG']['eqs_chg'].mean()*100, trades[trades['type']=='SHORT']['eqs_chg'].mean()*100
print(f' **************************************************************')
print(f'average returns: {x1:.1f}%  average returns (LONG): {x2:.1f} %, average returns (SHORT) {x3:.1f}% ')

 **************************************************************
average returns: 0.6%  average returns (LONG): 2.0 %, average returns (SHORT) 0.2% 


In [53]:
# DrawDowns
main_title = 'Draw-Downs 16-weeks-cycle, orginal pattern w & w/o fees'
sub_title = 'instrument: DAX [2000-2024]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

# calculations
x2 = pd.Series(eqs, index=price_w.index)
uw1 = ((pnl_full / pnl_full.expanding().max()) - 1) * 100
uw2 = ((x2 / x2.expanding().max()) - 1) * 100

# plotting
fig = go.Figure()
fig.update_layout(title=title)
fig.update_layout(autosize=False, width=1000, height=500)
fig.add_trace(go.Scatter(x=uw1.index, y=uw1, mode='none', fill='tozeroy', fillcolor='rgba(255, 255, 0, 0.3)', name='orig. pattern w/o fee'))
fig.add_trace(go.Scatter(x=uw2.index, y=uw2, mode='none', fill='tozeroy', fillcolor='rgba(0, 255, 0, 0.3)', name='orig. pattern w fee'))
fig.update_layout(template = 'plotly_dark', xaxis_title="Date", yaxis_title='drawdown (%)')
fig.show()

x1, x2 = uw1.min(), uw2.min()
print(f'*************************************************')
print(f'max DrawDown orig. pattern w/o fee: {x1:.1f}%   max DrawDown orig. pattern w fee: {x2:.1f}%    ')

*************************************************
max DrawDown orig. pattern w/o fee: -17.3%   max DrawDown orig. pattern w fee: -27.5%    


In [54]:
# gantt-chart of trading positions of various patterns
main_title = 'Position for 16-weeks-cycle, original pattern'
sub_title = 'instrument: DAX [2000-2024]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

various_trades = [trades]
labels = ['original pattern']
head_line = 'positions during the overall timeline (16-weeks-cycle), DAX [2000-2024]'
position_gantt(various_trades, labels, head_line)

#5. Sensitivity Analyses

Testing, how sensitive are the results regarding shifting the pattern within the 16 weeks cycle

In [56]:
# preparation of results dataframe, weekly data and setting parameters
result = pd.DataFrame(columns=['shift_wks','strategy'])
pos = pos_orig
weeks = 16
pnls=[]
for shift_weeks in range (weeks):
  # calculation of week-type and resulting positions depending on the shift
  pnl = pnl_accumulation(price_w,
                         pos_weekly(price_w, pos_orig, shift_weeks))
  pnls.append(pnl)
  result.loc[len(result.index)] = [shift_weeks, pnl[-1]]

In [59]:
# plotting results
main_title = 'PnL of 16-weeks-cycle, original pattern (w/0 fee)'
sub_title = 'instrument: DAX [2000-2024]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = px.bar(result, x=result['shift_wks'], y=result['strategy'])
fig.add_hline(y=pnl_buh_full[-1], line_dash='dot', line_color='green', annotation_text='Buy and Hold PnL', annotation_position='top left')
fig.update_layout(title = title, template = 'plotly_dark', autosize=False,width=800,height=400)
fig.update_layout(xaxis_title='shift parameter [in wks]', yaxis_title='PnL')

In [ ]:
# PnL trajectories for various values of shift
main_title = 'Pnl-Curves for varios values of shift 16-weeks-cycle, original pattern'
sub_title = 'instrument: DAX [2000-2024]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = go.Figure()

for i in range(0,len(pnls)):
  p = pnls[i]
  lb = f'shift = {i}'
  fig.add_trace(go.Scatter(x=p.index, y=p, name=lb))
fig.update_layout(template = 'plotly_dark', autosize=False, width=800, height=800)
fig.update_layout(title = title,  xaxis_title="Date", yaxis_title='drawdown')
fig.show()

In [60]:
# preparation of results dataframe, weekly data and setting parameters
result = pd.DataFrame(columns=['shift_wks','strategy'])
pos = pos_orig
weeks = 16
eqss=[]

for shift_weeks in range (weeks):
  # calculation of week-type and resulting positions depending on the shift
  eqs = pnl_acc_real_equity(price_w,
                            pos_weekly(price_w, pos_orig, shift=shift_weeks),
                            start_equity, fee, fixed_fee)
  eqss.append(eqs)
  result.loc[len(result.index)] = [shift_weeks, eqs[-1]]

In [62]:
# plotting results
main_title = 'Final Equity of 16-weeks-cycle, original pattern (w fee)'
sub_title = 'instrument: DAX [2000-2024]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = px.bar(result, x=result['shift_wks'], y=result['strategy'])
fig.add_hline(y=pnl_buh_full[-1] * start_equity, line_dash='dot', line_color='green', annotation_text='Buy and Hold PnL', annotation_position='top left')
fig.update_layout(title = title, template = 'plotly_dark', autosize=False,width=800,height=400)
fig.update_layout(xaxis_title='shift parameter [in wks]', yaxis_title='final equity')